In [44]:
import numpy as np
from scipy.optimize import minimize

# function
def f(x, Q, b):
    return 0.5 * x.T @ Q @ x - b.T @ x

# gradient
def grad_f(x, Q, b):
    return Q @ x - b

# SR1 update step for B
def sr1_update(B, s, y):
    B = B + np.outer(y - B @ s, y - B @ s) / ((y - B @ s).T @ s)
    return B

# SR1 algo
def sr1(Q, b, x0, B0, delta0=1.0, eta=2e-4, r=0.5, epsilon=1e-4, max_iter=1000):
    x = x0
    B = B0
    delta = delta0
    k = 0
    while np.linalg.norm(grad_f(x, Q, b)) > epsilon and k < max_iter:
        print(f'\nIteration {k}:')
        print('-----------------------------------------------------')
        print(f'x_{k} = {x}')
        print(f'B_{k} = {B}')
        # subproblem
        res = minimize(lambda s: grad_f(x, Q, b).T @ s + 0.5 * s.T @ B @ s, x0=np.zeros_like(x), bounds=[(-delta, delta)]*len(x))
        s = res.x
        print(f's_{k} = {s}')
        y = grad_f(x + s, Q, b) - grad_f(x, Q, b)
        print(f'y_{k} = {y}')

        ared = f(x, Q, b) - f(x + s, Q, b)
        pred = -(grad_f(x, Q, b).T @ s + 0.5 * s.T @ B @ s)

        print(f'ared = {ared}, pred = {pred}')

        if ared / pred > eta:
            x = x + s
        if ared / pred > 0.75:
            if np.linalg.norm(s) <= 0.8 * delta:
                # same delta
                delta = delta
            else:
                # double delta
                delta = 2 * delta
        elif 0.1 < ared / pred <= 0.75:
            # same delta
            delta = delta
        else:
            # delta half
            delta = 0.5 * delta

        B = sr1_update(B, s, y)

        print(f'B_{k+1} = {B}')

        k += 1

    return x

def Q_generator(n):
    Q = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            Q[i, j] = 1 / (i + j + 1)
    return Q

Q = Q_generator(5)
b = np.ones(5)
x0 = np.zeros(5)
B0 = np.eye(5)

x_opt = sr1(Q, b, x0, B0)
print(f'Optimal solution found @: {x_opt}')



Iteration 0:
-----------------------------------------------------
x_0 = [0. 0. 0. 0. 0.]
B_0 = [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]
s_0 = [1. 1. 1. 1. 1.]
y_0 = [2.28333332 1.44999999 1.09285714 0.88452381 0.74563492]
ared = 1.7718254041071422, pred = 2.4999999999999996
B_1 = [[ 2.13087193  0.39653951  0.08182561 -0.10175749 -0.22414623]
 [ 0.39653951  1.13904632  0.0286921  -0.0356812  -0.07859673]
 [ 0.08182561  0.0286921   1.00592059 -0.00736279 -0.01621837]
 [-0.10175749 -0.0356812  -0.00736279  1.00915629  0.020169  ]
 [-0.22414623 -0.07859673 -0.01621837  0.020169    1.04442725]]

Iteration 1:
-----------------------------------------------------
x_1 = [1. 1. 1. 1. 1.]
B_1 = [[ 2.13087193  0.39653951  0.08182561 -0.10175749 -0.22414623]
 [ 0.39653951  1.13904632  0.0286921  -0.0356812  -0.07859673]
 [ 0.08182561  0.0286921   1.00592059 -0.00736279 -0.01621837]
 [-0.10175749 -0.0356812  -0.00736279  1.00915629  0.020169  ]
 [

In [48]:
# DFP algo
def dfp(Q, b, x0, B0, delta0=1.0, eta=2e-4, r=0.5, epsilon=1e-4, max_iter=1000):
    x = x0
    B = B0
    delta = delta0
    k = 0
    while np.linalg.norm(grad_f(x, Q, b)) > epsilon and k < max_iter:
        print(f'\nIteration {k}:')
        print('-----------------------------------------------------')
        print(f'x_{k} = {x}')
        print(f'B_{k} = {B}')
        # subproblem
        res = minimize(lambda s: grad_f(x, Q, b).T @ s + 0.5 * s.T @ B @ s, x0=np.zeros_like(x), bounds=[(-delta, delta)]*len(x))
        s = res.x
        print(f's_{k} = {s}')
        y = grad_f(x + s, Q, b) - grad_f(x, Q, b)
        print(f'y_{k} = {y}')

        ared = f(x, Q, b) - f(x + s, Q, b)
        pred = -(grad_f(x, Q, b).T @ s + 0.5 * s.T @ B @ s)

        print(f'ared = {ared}, pred = {pred}')

        if ared / pred > eta:
            x = x + s
        if ared / pred > 0.75:
            if np.linalg.norm(s) <= 0.8 * delta:
                # same delta
                delta = delta
            else:
                # double delta
                delta = 2 * delta
        elif 0.1 < ared / pred <= 0.75:
            # same delta
            delta = delta
        else:
            # delta half
            delta = 0.5 * delta

        B = B + np.outer(y, y) / (y.T @ s) - B @ np.outer(s, s) @ B / (s.T @ B @ s)

        print(f'B_{k+1} = {B}')

        k += 1

    return x

# test for n=2
Q = np.array([[1, 0], [0, 2]])
b = np.array([1, 1])
x0 = np.zeros(2)
B0 = np.eye(2)

x_opt = dfp(Q, b, x0, B0)
print(f'Optimal solution found @: {x_opt}')



Iteration 0:
-----------------------------------------------------
x_0 = [0. 0.]
B_0 = [[1. 0.]
 [0. 1.]]
s_0 = [1. 1.]
y_0 = [1.         1.99999999]
ared = 0.500000005, pred = 1.0
B_1 = [[0.83333333 0.16666667]
 [0.16666667 1.83333333]]

Iteration 1:
-----------------------------------------------------
x_1 = [1. 1.]
B_1 = [[0.83333333 0.16666667]
 [0.16666667 1.83333333]]
s_1 = [ 0.11111068 -0.55555235]
y_1 = [ 0.11111068 -1.1111047 ]
ared = 0.24074113955595766, pred = 0.2777777727685321
B_2 = [[ 0.85294125 -0.02941181]
 [-0.02941181  1.99411763]]

Iteration 2:
-----------------------------------------------------
x_2 = [1.11111068 0.44444765]
B_2 = [[ 0.85294125 -0.02941181]
 [-0.02941181  1.99411763]]
s_2 = [-0.12841175  0.05382635]
y_2 = [-0.12841175  0.1076527 ]
ared = 0.0091062128376308, pred = 0.010123910057757283
B_3 = [[ 0.98321288 -0.04004847]
 [-0.04004847  1.90445767]]

Iteration 3:
-----------------------------------------------------
x_3 = [0.98269893 0.498274  ]
B_3 = 